# 1. Instalação e Configuração do Ambiente

Esta seção realiza a instalação dos pacotes necessários, como Selenium e o driver do Chrome, além de configurar o ambiente para rodar o código de coleta de dados.

In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install ace_tools

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 128 kB in 2s (68.6 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.ill

#Bairros

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from collections import Counter
import folium

def contar_farmacias(localidade):
    # Configurando o Selenium
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920x1080")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://www.google.com/maps")
    print(f"🌍 Página do Google Maps carregada para {localidade}.")

    # Pesquisa no Google Maps
    search_box = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "searchboxinput"))
    )
    search_box.send_keys(f"farmácias em {localidade}")
    search_box.send_keys(Keys.ENTER)

    time.sleep(10)
    scrollable_div = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]'))
    )
    print("⏳ Resultados carregados após a pesquisa.")

    # Scroll progressivo
    previous_count = 0
    farmacias_nomes = []  # Usar uma lista para permitir duplicatas

    while True:
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(3)

        current_count = len(driver.find_elements(By.CLASS_NAME, "Nv2PK"))
        if current_count == previous_count:
            break
        previous_count = current_count

    print("📊 Coletando os resultados após rolagem...")

    # Buscar todos os resultados
    results = driver.find_elements(By.CSS_SELECTOR, ".Nv2PK .qBF1Pd")
    for result in results:
        try:
            nome = result.text.strip()  # Remover espaços em branco
            farmacias_nomes.append(nome)  # Adicionar à lista para permitir duplicatas
        except Exception as e:
            print(f"⚠️ Erro ao tentar obter nome da farmácia: {e}")

    # Fecha o navegador
    driver.quit()

    return farmacias_nomes

# Lista de bairros de São Paulo para buscar farmácias
bairros = [
    # Zona Sul
    "Moema", "Paraíso", "Itaim Bibi", "Vila Mariana", "Campo Belo", "Brooklin", "Jabaquara", "Campo Grande",
    "Cidade Ademar", "Interlagos", "Jardim São Luís", "Santo Amaro", "Socorro", "Capela do Socorro",
    "Cidade Dutra", "Pedreira", "Grajaú", "Parelheiros", "Marsilac", "Vila Andrade",
    "Ipiranga", "Sacomã", "Cursino", "Capão Redondo", "Jardim Ângela", "Campo Limpo",

    # Zona Norte
    "Santana", "Tremembé", "Tucuruvi", "Casa Verde", "Vila Maria", "Vila Guilherme", "Jaçanã", "Mandaqui",
    "Freguesia do Ó", "Brasilândia", "Cachoeirinha", "Jaraguá", "Perus", "Anhanguera",
    "Limão", "Vila Nova Cachoeirinha",

    # Zona Leste
    "Vila Prudente", "Tatuapé", "Mooca", "Aricanduva", "Penha", "Ermelino Matarazzo", "Vila Matilde",
    "Artur Alvim", "Itaquera", "São Mateus", "São Miguel Paulista", "Vila Formosa", "Parque do Carmo",
    "Cidade Líder", "Vila Jacuí", "Cidade Tiradentes", "Guaianases", "Lajeado", "Água Rasa", "José Bonifácio",
    "São Rafael", "Jardim Helena", "Vila Curuçá",

    # Zona Oeste
    "Pinheiros", "Perdizes", "Butantã", "Lapa", "Vila Leopoldina", "Vila Madalena", "Rio Pequeno",
    "Jardim Bonfiglioli", "Raposo Tavares", "Jardim Paulista", "Jardim Europa", "Sumaré",
    "Jaguaré", "Morumbi", "Vila Sônia", "Jardim das Vertentes",

    # Centro
    "República", "Sé", "Consolação", "Liberdade", "Bela Vista", "Santa Cecília", "Bom Retiro",
    "Cambuci", "Brás", "Pari", "Armênia", "Barra Funda", "Campos Elíseos"
]



# Dicionário para armazenar o total de farmácias por bairro
farmacias_por_bairro = {}

# Buscar farmácias em todos os bairros
for index, bairro in enumerate(bairros, start=1):
    print(f"🔄 Processando bairro {index} de {len(bairros)}: {bairro}...")
    farmacias = contar_farmacias(bairro)
    farmacias_por_bairro[bairro] = farmacias

# Consolidar todas as farmácias e contar total por bairro
todas_farmacias = []

# Ordenar o total de farmácias por bairro em ordem decrescente
print("\n📊 Resumo total de farmácias por bairro (em ordem decrescente):")
bairros_ordenados = sorted(farmacias_por_bairro.items(), key=lambda item: len(item[1]), reverse=True)

# Imprimir o total de farmácias por bairro em ordem decrescente e numerado
for i, (bairro, farmacias) in enumerate(bairros_ordenados, start=1):
    total_bairro = len(farmacias)
    print(f"{i}. 📍 {bairro}: {total_bairro} farmácias")
    todas_farmacias.extend(farmacias)

# Contar a frequência de cada farmácia
nomes_farmacias = [farmacia['nome'] for farmacia in todas_farmacias if isinstance(farmacia, dict) and 'nome' in farmacia]
contagem_farmacias = Counter(nomes_farmacias)

# Resultados
total_farmacias = len(contagem_farmacias)
farmacia_mais_frequente = contagem_farmacias.most_common(1)[0] if contagem_farmacias else (None, 0)

print(f"\n📈 Total de farmácias encontradas em todos os bairros: {total_farmacias}")
print(f"🏥 Farmácia mais frequente: {farmacia_mais_frequente[0]} - Apareceu {farmacia_mais_frequente[1]} vezes.")

🔄 Processando bairro 1 de 94: Moema...
🌍 Página do Google Maps carregada para Moema.
⏳ Resultados carregados após a pesquisa.
📊 Coletando os resultados após rolagem...
🔄 Processando bairro 2 de 94: Paraíso...
🌍 Página do Google Maps carregada para Paraíso.
⏳ Resultados carregados após a pesquisa.
📊 Coletando os resultados após rolagem...
🔄 Processando bairro 3 de 94: Itaim Bibi...
🌍 Página do Google Maps carregada para Itaim Bibi.
⏳ Resultados carregados após a pesquisa.
📊 Coletando os resultados após rolagem...
🔄 Processando bairro 4 de 94: Vila Mariana...
🌍 Página do Google Maps carregada para Vila Mariana.
⏳ Resultados carregados após a pesquisa.
📊 Coletando os resultados após rolagem...
🔄 Processando bairro 5 de 94: Campo Belo...
🌍 Página do Google Maps carregada para Campo Belo.
⏳ Resultados carregados após a pesquisa.
📊 Coletando os resultados após rolagem...
🔄 Processando bairro 6 de 94: Brooklin...
🌍 Página do Google Maps carregada para Brooklin.
⏳ Resultados carregados após a 

In [ ]:
# Consolidar todas as farmácias e contar total por bairro
todas_farmacias = []

# Ordenar o total de farmácias por bairro em ordem decrescente
print("\n📊 Resumo total de farmácias por bairro (em ordem decrescente):")
bairros_ordenados = sorted(farmacias_por_bairro.items(), key=lambda item: len(item[1]), reverse=True)

# Imprimir o total de farmácias por bairro em ordem decrescente e numerado
for i, (bairro, farmacias) in enumerate(bairros_ordenados, start=1):
    total_bairro = len(farmacias)
    print(f"{i}. 📍 {bairro}: {total_bairro} farmácias")
    todas_farmacias.extend(farmacias)

# Contar a frequência de cada farmácia
contagem_farmacias = Counter(todas_farmacias)

# Resultados - Top 3 farmácias mais frequentes
top_3_farmacias = contagem_farmacias.most_common(3)

print(f"\n📈 Total de farmácias encontradas em todos os bairros: {len(todas_farmacias)}")
print(f"🏆 Top 3 farmácias mais frequentes:")

for i, (farmacia, frequencia) in enumerate(top_3_farmacias, start=1):
    print(f"{i}. {farmacia} - Apareceu {frequencia} vezes.")

# Analisando a quantidade de farmácias por zona
zonas = {
    "Zona Sul": [
        "Moema", "Paraíso", "Itaim Bibi", "Vila Mariana", "Campo Belo", "Brooklin", "Jabaquara", "Campo Grande",
        "Cidade Ademar", "Interlagos", "Jardim São Luís", "Santo Amaro", "Socorro", "Capela do Socorro",
        "Cidade Dutra", "Pedreira", "Grajaú", "Parelheiros", "Marsilac", "Vila Andrade", "Ipiranga",
        "Sacomã", "Cursino", "Capão Redondo", "Jardim Ângela", "Campo Limpo"
    ],
    "Zona Norte": [
        "Santana", "Tremembé", "Tucuruvi", "Casa Verde", "Vila Maria", "Vila Guilherme", "Jaçanã", "Mandaqui",
        "Freguesia do Ó", "Brasilândia", "Cachoeirinha", "Jaraguá", "Perus", "Anhanguera", "Limão",
        "Vila Nova Cachoeirinha"
    ],
    "Zona Leste": [
        "Vila Prudente", "Tatuapé", "Mooca", "Aricanduva", "Penha", "Ermelino Matarazzo", "Vila Matilde",
        "Artur Alvim", "Itaquera", "São Mateus", "São Miguel Paulista", "Vila Formosa", "Parque do Carmo",
        "Cidade Líder", "Vila Jacuí", "Cidade Tiradentes", "Guaianases", "Lajeado", "Água Rasa",
        "José Bonifácio", "São Rafael", "Jardim Helena", "Vila Curuçá"
    ],
    "Zona Oeste": [
        "Pinheiros", "Perdizes", "Butantã", "Lapa", "Vila Leopoldina", "Vila Madalena", "Rio Pequeno",
        "Jardim Bonfiglioli", "Raposo Tavares", "Jardim Paulista", "Jardim Europa", "Sumaré", "Jaguaré",
        "Morumbi", "Vila Sônia", "Jardim das Vertentes"
    ],
    "Centro": [
        "República", "Sé", "Consolação", "Liberdade", "Bela Vista", "Santa Cecília", "Bom Retiro",
        "Cambuci", "Brás", "Pari", "Armênia", "Barra Funda", "Campos Elíseos"
    ]
}

# Contabilizar a quantidade de farmácias por zona
total_farmacias_zona = {}

for zona, bairros_zona in zonas.items():
    total_farmacias_zona[zona] = sum(len(farmacias_por_bairro.get(bairro, [])) for bairro in bairros_zona)

# Encontrar a zona com mais farmácias
zona_mais_farmacias = max(total_farmacias_zona.items(), key=lambda item: item[1])

# Resultados sobre zonas
print("\n📊 Resumo total de farmácias por zona:")
for zona, total in total_farmacias_zona.items():
    print(f"🔹 {zona}: {total} farmácias")

print(f"\n🏆 Zona com mais farmácias: {zona_mais_farmacias[0]} - Total: {zona_mais_farmacias[1]} farmácias")



📊 Resumo total de farmácias por bairro (em ordem decrescente):
1. 📍 Vila Mariana: 120 farmácias
2. 📍 Campo Grande: 120 farmácias
3. 📍 Santana: 120 farmácias
4. 📍 Jaraguá: 120 farmácias
5. 📍 São Mateus: 120 farmácias
6. 📍 São Rafael: 120 farmácias
7. 📍 Jardim Paulista: 120 farmácias
8. 📍 Sumaré: 120 farmácias
9. 📍 Bela Vista: 120 farmácias
10. 📍 Guaianases: 110 farmácias
11. 📍 Anhanguera: 107 farmácias
12. 📍 Sacomã: 106 farmácias
13. 📍 Tatuapé: 106 farmácias
14. 📍 Ipiranga: 104 farmácias
15. 📍 Parque do Carmo: 100 farmácias
16. 📍 Mooca: 99 farmácias
17. 📍 Vila Prudente: 95 farmácias
18. 📍 Moema: 94 farmácias
19. 📍 Cachoeirinha: 93 farmácias
20. 📍 Interlagos: 92 farmácias
21. 📍 Vila Andrade: 91 farmácias
22. 📍 Perdizes: 91 farmácias
23. 📍 Pinheiros: 90 farmácias
24. 📍 Campo Belo: 88 farmácias
25. 📍 Lajeado: 88 farmácias
26. 📍 Santo Amaro: 87 farmácias
27. 📍 Brooklin: 86 farmácias
28. 📍 Mandaqui: 82 farmácias
29. 📍 Morumbi: 82 farmácias
30. 📍 Cidade Dutra: 81 farmácias
31. 📍 Jardim São L

In [ ]:
import folium
from folium.plugins import HeatMap

# Coordenadas centrais aproximadas para cada bairro de São Paulo (latitude, longitude)
coordenadas_bairros = {
    # Zona Sul
    "Moema": (-23.6015, -46.6655),
    "Paraíso": (-23.5733, -46.6446),
    "Itaim Bibi": (-23.5824, -46.6884),
    "Vila Mariana": (-23.5895, -46.6347),
    "Campo Belo": (-23.6265, -46.6749),
    "Brooklin": (-23.6089, -46.6970),
    "Jabaquara": (-23.6489, -46.6420),
    "Campo Grande": (-23.6511, -46.6702),
    "Cidade Ademar": (-23.6672, -46.6611),
    "Interlagos": (-23.6978, -46.6824),
    "Jardim São Luís": (-23.6663, -46.7625),
    "Santo Amaro": (-23.6543, -46.7157),
    "Socorro": (-23.6743, -46.7239),
    "Capela do Socorro": (-23.7097, -46.7463),
    "Cidade Dutra": (-23.7075, -46.6861),
    "Pedreira": (-23.6751, -46.6739),
    "Grajaú": (-23.7502, -46.7228),
    "Parelheiros": (-23.8236, -46.7381),
    "Marsilac": (-23.8789, -46.7836),
    "Vila Andrade": (-23.6090, -46.7158),
    "Ipiranga": (-23.5883, -46.6106),
    "Sacomã": (-23.6345, -46.5963),
    "Cursino": (-23.6156, -46.6134),

    # Zona Norte
    "Santana": (-23.5002, -46.6245),
    "Tremembé": (-23.4616, -46.6097),
    "Tucuruvi": (-23.4782, -46.6073),
    "Casa Verde": (-23.5127, -46.6618),
    "Vila Maria": (-23.4977, -46.5894),
    "Vila Guilherme": (-23.5112, -46.5943),
    "Jaçanã": (-23.4605, -46.5770),
    "Mandaqui": (-23.4822, -46.6246),
    "Freguesia do Ó": (-23.5006, -46.6937),
    "Brasilândia": (-23.4820, -46.6983),
    "Cachoeirinha": (-23.4906, -46.6752),
    "Jaraguá": (-23.4575, -46.7444),
    "Perus": (-23.4049, -46.7547),
    "Anhanguera": (-23.3948, -46.7699),

    # Zona Leste
    "Vila Prudente": (-23.5803, -46.5857),
    "Tatuapé": (-23.5406, -46.5753),
    "Mooca": (-23.5549, -46.5895),
    "Aricanduva": (-23.5487, -46.5072),
    "Penha": (-23.5371, -46.5356),
    "Ermelino Matarazzo": (-23.5011, -46.4701),
    "Vila Matilde": (-23.5394, -46.5146),
    "Artur Alvim": (-23.5411, -46.5024),
    "Itaquera": (-23.5284, -46.4691),
    "São Mateus": (-23.6043, -46.4631),
    "São Miguel Paulista": (-23.4993, -46.4375),
    "Vila Formosa": (-23.5635, -46.5475),
    "Parque do Carmo": (-23.5694, -46.4532),
    "Cidade Líder": (-23.5638, -46.4873),
    "Vila Jacuí": (-23.4924, -46.4096),
    "Cidade Tiradentes": (-23.6031, -46.4042),
    "Guaianases": (-23.5336, -46.4115),
    "Lajeado": (-23.5173, -46.3983),
    "Água Rasa": (-23.5587, -46.5654),

    # Zona Oeste
    "Pinheiros": (-23.5614, -46.6955),
    "Perdizes": (-23.5389, -46.6667),
    "Butantã": (-23.5732, -46.7133),
    "Lapa": (-23.5250, -46.7035),
    "Vila Leopoldina": (-23.5275, -46.7258),
    "Jaguaré": (-23.5488, -46.7525),
    "Morumbi": (-23.6021, -46.7219),
    "Vila Sônia": (-23.5850, -46.7192),
    "Rio Pequeno": (-23.5687, -46.7418),
    "Raposo Tavares": (-23.5918, -46.7587),

    # Centro
    "República": (-23.5475, -46.6333),
    "Sé": (-23.5505, -46.6333),
    "Consolação": (-23.5536, -46.6527),
    "Liberdade": (-23.5587, -46.6328),
    "Bela Vista": (-23.5562, -46.6466),
    "Santa Cecília": (-23.5382, -46.6497),
    "Bom Retiro": (-23.5283, -46.6385),
    "Cambuci": (-23.5747, -46.6147),
    "Brás": (-23.5446, -46.6092),
    "Pari": (-23.5172, -46.6174),
    "Armênia": (-23.5306, -46.6237)
}


# Consolidar os dados de farmácias por bairro em um formato que possamos usar no mapa de calor
heat_data = []

for bairro, coordenadas in coordenadas_bairros.items():
    total_farmacias = len(farmacias_por_bairro.get(bairro, []))
    if total_farmacias > 0:
        # Adiciona a coordenada do bairro e o peso de acordo com o número de farmácias
        heat_data.append([coordenadas[0], coordenadas[1], total_farmacias])

# Criar um mapa de São Paulo usando Folium
mapa_calor = folium.Map(location=[-23.5505, -46.6333], zoom_start=11)

# Adicionar o HeatMap ao mapa
HeatMap(heat_data).add_to(mapa_calor)

# Salvar e exibir o mapa
mapa_calor.save("mapa_calor_farmacias_sao_paulo.html")
print("\n🗺️ Mapa de calor gerado e salvo como 'mapa_calor_farmacias_sao_paulo.html'. Abra este arquivo em um navegador para visualizar.")

# Se estiver usando Jupyter Notebook, você pode exibir o mapa diretamente:
from IPython.display import display
display(mapa_calor)



🗺️ Mapa de calor gerado e salvo como 'mapa_calor_farmacias_sao_paulo.html'. Abra este arquivo em um navegador para visualizar.
